In [ ]:
from gensim.models import KeyedVectors
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import numpy as np

In [ ]:
path = '/content/drive/My Drive/data/'
w2v = KeyedVectors.load_word2vec_format(path+'GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
tokenizer = Tokenizer()
vocab_size = 427
MAX_LEN = 13

embedding_matrix = np.zeros((vocab_size, 300))
model = load_model(path+'benchmark/model.h5')
total_class = ["forward", "backward", "right", "left"]

In [ ]:
def predict_cmd(cmd):
  cmd = np.array([cmd])
  tokenizer.fit_on_texts(cmd)
  cmd = tokenizer.texts_to_sequences(cmd)
  cmd = pad_sequences(cmd, padding='post', maxlen=MAX_LEN)

  bugCase = {'to':'To','a':'A','and':'And','of':'Of','flavour':'Flavour','counselling':'Counselling','whitethorn':'Whitethorn','plough':'Plough','favour':'Favour'}
  not_found = 1
  for word, index in tokenizer.word_index.items():
      try:
        if word in bugCase.keys():
          word = bugCase[word]
        embedding_vector = w2v[word]
        if embedding_vector is not None:
          embedding_matrix[index] = embedding_vector
      except:
        print(not_found,word)
        not_found += 1

  model.layers[1].set_weights([embedding_matrix])
  res = model.predict(cmd)
  return (total_class[np.argmax(res)], res[0][np.argmax(res)]*100)

In [19]:
command = 'go away from here'
res = predict_cmd(command)
print(command,'=> '+res[0]+' '+str(round(res[1]))+'%')
command = 'move to your right side'
res = predict_cmd(command)
print(command,'=> '+res[0]+' '+str(round(res[1]))+'%')
command = 'hey look at your left side'
res = predict_cmd(command)
print(command,'=> '+res[0]+' '+str(round(res[1]))+'%')
command = 'come on buddy get closer'
res = predict_cmd(command)
print(command,'=> '+res[0]+' '+str(round(res[1]))+'%')
command = 'flowers are beautiful'
res = predict_cmd(command)
print(command,'=> '+res[0]+' '+str(round(res[1]))+'%')
command = 'watermelon is larger than apple'
res = predict_cmd(command)
print(command,'=> '+res[0]+' '+str(round(res[1]))+'%')
command = 'follow me to the door'
res = predict_cmd(command)
print(command,'=> '+res[0]+' '+str(round(res[1]))+'%')
command = 'right now move to your left'
res = predict_cmd(command)
print(command,'=> '+res[0]+' '+str(round(res[1]))+'%')

go away from here => backward 100.0%
move to your right side => right 100.0%
hey look at your left side => left 100.0%
come on buddy get closer => forward 100.0%
flowers are beautiful => forward 96.0%
watermelon is larger than apple => forward 73.0%
follow me to the door => forward 100.0%
right now move to your left => right 50.0%


In [6]:
par = 'go straight ahead.then move to your left.find the glass and grab it.if required move a bit forward to grab it.then turn to your right.pour the water on ground'
sen = par.split('.')
for s in sen:
  res = predict_cmd(s)
  print(s,'=> '+res[0]+' '+str(round(res[1]))+'%')

go straight ahead => forward 100.0%
then move to your left => left 100.0%
find the glass and grab it => backward 45.0%
if required move a bit forward to grab it => forward 100.0%
then turn to your right => right 100.0%
pour the water on ground => left 44.0%
